# 拉普拉斯方法定轨道六根数

## 1. 已知三次观测的观测坐标，定出某一时刻的位置矢量和速度矢量
按照笔记的方法分别推出各个量

注意时间单位的选取!

In [5]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime
import time

import DDS_CMpack as DDS

%matplotlib inline

In [6]:
# astronomy constants
R_earth = 6371
au = 149597870
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

time_unit = 806.81163 #806.8116 # SI 


In [7]:
# give 3 observed position
# position_fram = pd.DataFrame({'alpha':\
#                         np.array([142.935,157.274166667,171.817916667,183.134166667,\
#                                   208.640416667,219.102083333]),\
#                        'delta':\
#                        np.array([8.521111111,-2.395,-14.508333333,-23.56333333,-40.020277777,-44.998611111]),\
#                        'UT1_hours':np.array([21.575128333,21.60311055,21.631419722,\
#                                              21.654491388,21.714094166,21.742875555])})

position_fram = pd.read_csv('Echo1st.csv')

position_fram

,UT1_hours,alpha,delta,year,month,days
0,21.575128,142.935000,8.521111,1965,1,14
1,21.603111,157.274167,-2.395000,1965,1,14
2,21.631420,171.817917,-14.508333,1965,1,14
3,21.654491,183.134167,-23.563333,1965,1,14
4,21.714094,208.640417,-40.020278,1965,1,14
5,21.742876,219.102083,-44.998611,1965,1,14


In [8]:
# 计算观测数据的儒略日时间和恒星时
year = 1965; month = 1; days = 14;
UT1_hours = np.array([21.575128333,21.60311055,21.631419722,21.654491388,21.714094166,21.742875555])
jd_t = []
SG_t = []
for i in range(len(UT1_hours)):
    SG_s,jd_s = DDS.UTC2SG(year,month,days,UT1_hours[i])
    SG_t.append(SG_s); jd_t.append(jd_s);
    
SG_array = np.array(SG_t); 
SG_degree = SG_array%(3600*24)/(3600*24)*360

jd_array = np.array(jd_t)
delta_t = (UT1_hours - UT1_hours[0])*3600/time_unit

position_fram['deltaT'] = delta_t

delta_t3

OverflowError: mktime argument out of range

In [9]:
# give the station's lam and phi
lam = 118.82091666
phi = 31.893611111


In [7]:
# 需要根据alpha和delta求出所需的测站到地心的RA,DEC
station_earth = []
for i in range(len(position_fram)):
    # give the station's RA and DEC then convert to station_earth
    RA_station = SG_degree[i] + lam
    DEC_station = phi
    station_earth.append(DDS.RADEC2xyz(r_station_earth,RA_station,DEC_station))

station_earth

[array([-0.81183633, -0.24592818,  0.52786921]),
 array([-0.81000791, -0.25188507,  0.52786921]),
 array([-0.80811365, -0.25789773,  0.52786921]),
 array([-0.80653683, -0.26278746,  0.52786921]),
 array([-0.80232652, -0.27537441,  0.52786921]),
 array([-0.80022302, -0.28142853,  0.52786921])]

In [8]:
position_fram

,UT1_hours,alpha,delta,year,month,days,deltaT
0,21.575128,142.935000,8.521111,1965,1,14,0.000000
1,21.603111,157.274167,-2.395000,1965,1,14,0.124857
2,21.631420,171.817917,-14.508333,1965,1,14,0.251173
3,21.654491,183.134167,-23.563333,1965,1,14,0.354119
4,21.714094,208.640417,-40.020278,1965,1,14,0.620067
5,21.742876,219.102083,-44.998611,1965,1,14,0.748489


In [9]:
# add Pj ,Qj to positon_fram_fil
Lt = []; Pt = []; Qt = []
Lambda = []; Miu = []; Niu = [];

#由 delta,alpha 算出 L = [Lambda,Miu,Niu]
for j in range(len(position_fram)):
    delta = position_fram['delta'].values[j]*2*np.pi/360 ; 
    alpha = position_fram['alpha'].values[j]*2*np.pi/360;

    L = [np.cos(delta)*np.cos(alpha),\
         np.cos(delta)*np.sin(alpha),\
         np.sin(delta)]
    
    Lambda.append(L[0]); 
    Miu.append(L[1]);
    Niu.append(L[2]);
    
#     Lambda.append(np.cos(delta)*np.cos(alpha))
#     Miu.append(np.cos(delta)*np.sin(alpha))
#     Niu.append(np.sin(delta))

# 由[Lambda,Miu,Niu] 和 测站-地心矢量归算出 P,Q
    Pj = L[2]*station_earth[j][0] - L[0]*station_earth[j][2]
    Qj = L[2]*station_earth[j][1] - L[1]*station_earth[j][2]
    
    Lt.append(L)
    Pt.append(Pj)
    Qt.append(Qj)
#     print(Lt,Pt,Qt)

position_fram_fil = pd.DataFrame({'alpha':\
                        np.array([142.935,157.274166667,171.817916667,183.134166667,\
                                  208.640416667,219.102083333]),\
                       'delta':\
                       np.array([8.521111111,-2.395,-14.508333333,-23.56333333,-40.020277777,-44.998611111]),\
                       'jd':jd_array,\
                        'SG_d': SG_degree,\
                        'deltaT':delta_t,\
                        'P':np.array(Pt),'Q':np.array(Qt),\
                        'Lambda':np.array(Lambda),\
                        'Miu':np.array(Miu),'Niu':np.array(Niu)})
    
Lt 

[[-0.7891439307993768, 0.5960674272555554, 0.14817381228628504],
 [-0.9215582970478929, 0.38598449379160416, -0.04178846365477428],
 [-0.9582566000732619, 0.13778102486286736, -0.25052081271577153],
 [-0.9152477093203553, -0.0501154444748697, -0.39976252051552885],
 [-0.6721154419353875, -0.36706452095992986, -0.6430586832976101],
 [-0.5487447629378402, -0.44598590319844417, -0.7070896402130082]]

In [10]:
position_fram_fil

,Lambda,Miu,Niu,P,Q,SG_d,alpha,delta,deltaT,jd
0,-0.789144,0.596067,0.148174,0.296272,-0.351086,78.032128,142.935000,8.521111,0.000000,-12769.600293
1,-0.921558,0.385984,-0.041788,0.520311,-0.193223,78.453011,157.274167,-2.395000,0.124857,-12769.599128
2,-0.958257,0.137781,-0.250521,0.708283,-0.008122,78.878811,171.817917,-14.508333,0.251173,-12769.597948
3,-0.915248,-0.050115,-0.399763,0.805554,0.131507,79.225833,183.134167,-23.563333,0.354119,-12769.596987
4,-0.672115,-0.367065,-0.643059,0.870732,0.370844,80.122323,208.640417,-40.020278,0.620067,-12769.594503
5,-0.548745,-0.445986,-0.707090,0.855495,0.434417,80.555226,219.102083,-44.998611,0.748489,-12769.593304


In [43]:
# 为循环重新format，取出需要的数据
Observe_time = len(position_fram)

obs = [0,1,2,3,4,5] # 取全部观测，做最小二乘法定初轨

# 取定t0 
index = 1
t0 = 0.15
# t0 = 0.5*(position_fram_fil['deltaT'].values[index] + position_fram_fil['deltaT'].values[index+2])
print(t0)
# set F0 and G0 to begin the loop

F0 = np.ones(Observe_time).tolist()
G0 = (position_fram_fil['deltaT'].values - t0).tolist()

FN0 = []; GN0 = []
P = []; Q = []
Lambda = []; Niu = []; Miu = []
tao = [];
for i in range(len(obs)): # 把要用的那几次观测数据提取出来：
    j = obs[i]
    FN0.append(F0[j])
    GN0.append(G0[j])
    P.append(position_fram_fil['P'].values[j])
    Q.append(position_fram_fil['Q'].values[j])
    Niu.append(position_fram_fil['Niu'].values[j])
    Miu.append(position_fram_fil['Miu'].values[j])
    Lambda.append(position_fram_fil['Lambda'].values[j])
    tao.append(position_fram_fil['deltaT'].values[j] - t0)
    

0.15


In [44]:
# 循环，解出F,G。

Nwind = 0
key = 0
F = [FN0]; G = [GN0]

while key == 0:
    
    # 在一次循环中:
    # 最终的目的是解(A,b)的增广矩阵 - 对应(x,y,z,vx,vy,vz)
    A = []
    b = []

    # we have : Lt, position_fram_fil
    ## 从F,G 解r0,v0
    for j in range(len(obs)): # 三次观测(现在的F,G已经只剩我们需要的那几次观测的值了)

        A.append([Niu[j]*F[Nwind][j],0,-1*Lambda[j]*F[Nwind][j],Niu[j]*G[Nwind][j],0,-1*Lambda[j]*G[Nwind][j]])
        A.append([0,Niu[j]*F[Nwind][j],-1*Miu[j]*F[Nwind][j],0,Niu[j]*G[Nwind][j],-1*Miu[j]*G[Nwind][j]])
        b.append(P[j])
        b.append(Q[j])

#################################################################        
    # 三次观测，解矩阵得到最终的结果
        
#     AA = np.array(A)
#     bb = np.array(b)
    
#     print(Nwind,b)
    
#     ans = np.linalg.solve(AA,bb) #ans 6 位对应(x,y,z,vx,vy,vz)

#     type(ans)
#     r0 = np.sqrt(ans[0]**2 + ans[1]**2 + ans[2]**2)
#     v0 = np.sqrt(ans[3]**2 + ans[4]**2 + ans[5]**2)
#     print(Nwind,':',r0,v0)
    
#################################################################
    # 多次测量、多资料问题的解法：
    
    AA = np.array(A)
    b = np.array(b)

#     print(np.dot(AA.T.copy(),AA))
#     print(np.dot(AA.T.copy(),b))

    ans = np.linalg.solve(np.dot(AA.T.copy(),AA),\
                             np.dot(AA.T.copy(),b))
    
    r0 = np.sqrt(ans[0]**2 + ans[1]**2 + ans[2]**2)
    v0 = np.sqrt(ans[3]**2 + ans[4]**2 + ans[5]**2)
    print(Nwind,':',r0,v0)
    
#################################################################
    # 算出新的F 和 G，为接下来的迭代做准备

    # r0,v0获得F,G
    FNw = []; GNw = [];
    for j in range(len(obs)):
#         print(j)
        FNw.append(DDS.F(r0,v0,tao[j]))
        GNw.append(DDS.G(r0,v0,tao[j]))
        
    F.append(FNw); G.append(GNw)

#################################################################
    # 判定deltaF or G的大小，并且决定最后是不是选取这一项。
    Nwind += 1
    delta_F = np.abs(np.array(F[Nwind]) - np.array(F[Nwind - 1]));
    delta_G = np.abs(np.array(G[Nwind]) - np.array(G[Nwind - 1]))
    big_delta = np.max([np.max(delta_F),np.max(delta_G)])
    
    print(delta_F,delta_G)
    print('--------------------')

    if big_delta < 1e-13:
        print('end')
        print(delta_F,delta_G)
        r0_a = np.array([ans[0],ans[1],ans[2]])
        v0_a = np.array([ans[3],ans[4],ans[5]])
        key = 1

# 地球半径是 6371 km

0 : 0.9982180659223927 0.047632915164560106
[0.01143453 0.0003182  0.00512951 0.02088817 0.11275097 0.18586702] [5.76758582e-04 2.66883846e-06 1.72989788e-04 1.42831737e-03
 1.83807692e-02 3.96085863e-02]
--------------------
1 : 1.5625034066825467 1.93164220790373
[0.00783425 0.00023268 0.0039341  0.01644753 0.09055942 0.14721951] [3.78813216e-04 1.94075206e-06 1.34910302e-04 1.15318400e-03
 1.49348423e-02 3.10564603e-02]
--------------------
2 : 1.098896544699752 0.42034168641496944
[0.00541579 0.00015501 0.00252285 0.01021022 0.05109722 0.07856276] [2.67812517e-04 1.29777294e-06 8.51892115e-05 6.93040134e-04
 7.70904972e-03 1.45736380e-02]
--------------------
3 : 1.3014914955322903 1.1209222410942044
[3.15993567e-03 9.39722208e-05 1.57961118e-03 6.54073072e-03
 3.44118833e-02 5.40625322e-02] [1.52455158e-04 7.84012584e-07 5.39949163e-05 4.54146704e-04
 5.43965225e-03 1.06456945e-02]
--------------------
4 : 1.147991639626326 0.6018260656142008
[2.22895743e-03 6.51730585e-05 1.07808

In [45]:
print(r0_a*R_earth,r0*R_earth,(r0-r_station_earth)*R_earth)

print(v0_a*R_earth/time_unit,v0*R_earth/time_unit)

print(t0*time_unit)

[-6840165.79265459 -1009615.79284533  3216480.04798529] 7625807.237220993 1260528.3952209929
[-1073.49321406 -4739.16395245 -3709.31951592] 6113.191794800064
121.0217445


In [46]:
r0_a*R_earth

array([-6840165.79265459, -1009615.79284533,  3216480.04798529])

In [47]:
v0_a*R_earth/time_unit

array([-1073.49321406, -4739.16395245, -3709.31951592])

## 2. 已知某一时刻的位置矢量和速度矢量，求轨道六根数

按照笔记方法,逐步求得：
a
n()
E()
e
M
i Omega w

In [48]:
# 从第二步开始，因为不需要用到F,G的计算，所以不用再取理论单位，现在取国际单位制
# astronomy constants
R_earth = 6371e3 # m

M_earth = 5.965e24 # kg
G_graviation = 6.672e-11 # N·m^2 /kg^2 

au = 149597870e3 # m
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

time_unit = 806.81163 #806.8116 # SI 

# miu_GM = 398600.5e-6 # km^3/SI^2

In [49]:
miu_GM = G_graviation*M_earth # 国际单位制

miu_GM

397984800000000.0

In [50]:
# input r0,v0,t0
r0 = r0_a*R_earth # unit= m
r0_norm = np.sqrt(sum([ i*i for i in r0])) # m

v0 = v0_a*R_earth/time_unit # unit= m/SI
v0_norm = np.sqrt(sum([ i*i for i in v0])) # m/SI

t0 = 121.0217445 # unit = SI from the first data we have
## t0 根据自己的单位去选择。

r0_norm/10**3,v0_norm/10**3

(7625.8072372209945, 6.113191794800063)

In [51]:
# get a 

a = 1/(2/r0_norm - v0_norm**2/miu_GM) # unit = m

a/R_earth

0.9322594973523285

In [52]:
# get n\E then a
n = np.sqrt(miu_GM/(a**3))

tan_E = (1 - r0_norm/a)*(a**2*n)/(np.dot(r0,v0))

e = np.sqrt( (1 - r0_norm/a)**2 + (np.dot(r0,v0) / (n*a**2))**2 )
# e = 0.01002

print(1 - r0_norm/a)
print(np.dot(r0,v0) / (n*a**2))

cos_E = (1 - r0_norm/a)/e
sin_E = (np.dot(r0,v0) / (a**2*n))/e

e

-0.28393016267397986
0.004044805026585536


0.28395897190223757

In [53]:
def arctan2(sinE,cosE):
    if sinE >=0 and cosE >=0: # I
        output = np.arcsin(sinE)
    elif sinE >=0 and cosE <=0: # II
        output = np.pi - np.arcsin(sinE)
    elif sinE <=0 and cosE >=0: # IV
        output = 2*np.pi + np.arcsin(sinE)
    else: # III
        output = np.pi - np.arcsin(sinE)
    return output


In [54]:
print(sin_E,cos_E,tan_E)

E = arctan2(sin_E,cos_E)
print(E)
np.degrees(E)

0.014244329029258833 -0.9998985443986337 -70.19625440726432
3.1273478428186614


179.18383246285165

In [55]:
# keplar 定 M
M = E - e*sin_E
M

3.123303037792076

In [56]:
# 定 i Omega w

#### method 1
P = (cos_E/r0_norm)*r0 - (sin_E/(a*n))*v0
Q = (sin_E/(r0_norm)*np.sqrt(1-e**2))*r0 + ((cos_E - e)/(a*n*np.sqrt(1-e**2)))*v0
R = np.cross(P,Q)

cos_i = R[2]
tan_Omega = -1*R[0]/R[1]

# get w ->[-90,+90]
tan_w = P[2]/Q[2]

############################################################

#### method 2
h = np.cross(r0,v0)
h_norm = np.sqrt(np.dot(h,h))

h_A, h_B, h_C = h[0:3]

# get cos_i, sin_i
cos_i_check = (h_C/h_norm)
tan_i = np.sqrt(h_A**2 + h_B**2) / h_C

sin_i = tan_i*cos_i_check


# get cos_\sin_Omega
sin_Omega = h_A/(h_norm*sin_i)
cos_Omega = -h_B/(h_norm*sin_i)

# finally get i Omega w

i = arctan2(sin_i,cos_i_check)

Omega = arctan2(sin_Omega,cos_Omega)

w = np.arctan(tan_w)

np.degrees(np.array([i,Omega,w]))

array([ 47.76872727,  33.37460807, -34.13810697])

## 3. 由轨道6根数归算星历表

已知：
* a, # m
* n(), # rad/s
* E(), # rad
* e, # Nan
* M, # rad
* [i, Omega, w] = P,Q # [rad,rad,rad]

t0 = 121.0217445 **# unit = SI from the first data we have**

M = M0 + n(t-t0)

M = E - esin(E)

In [105]:
print('a',a,'n',n,'E',E,'e',e,'M',M,i,Omega,w)

a 5939425.257631685 n 0.001378213852056141 E 3.1273478428186614 e 0.28395897190223757 M 3.123303037792076 0.833721570304591 0.5824967974093022 -0.5958223670843593


In [58]:
# 假设我们已知a,e,M,i,Omega,w,那么E,n均需要从六根数推出来。
E_old = E
n_old = n

n = np.sqrt(miu_GM/a**3)

n

0.001378213852056141

0.8599999999999999

In [106]:
# calculate keplar equation
# 设置初值t0 和我们需要归算的时间t

# 我将计算一整个周期的时间序列
T = 2*np.pi/n
print(T)

t = np.arange(0,T,1000) # unit = SI

Mt = (M + n*(t))%(2*np.pi)

print(t)
print(Mt)

4558.933504989647
[   0. 1000. 2000. 3000. 4000.]
[3.12330304 4.50151689 5.87973074 0.97475929 2.35297314]


In [107]:
# 迭代 开普勒方程解 E
Et = []
epsilon = 1e-6
for M_single in Mt:
    print(M_single)
    key = 0
    En0 = M_sinle
    Enw = [En0]
    Nwind = 0
    while key == 0:
        Enw.append(M_single + e*np.sin(Enw[Nwind]))

        Nwind += 1
#         print(Enw[Nwind])

        delta_Enw = np.abs(Enw[Nwind] - Enw[Nwind - 1])
#         print(delta_Enw)
#         print('------------')

        if delta_Enw < epsilon:
            key = 1;

    E_single = Enw[Nwind]
    Et.append(E_single)
print('E:',Et)

3.123303037792076
4.501516889848217
5.879730741904359
0.9747592867809125
2.3529731388370543
E: [3.1273476243507092, 4.247672202300532, 5.730710613250116, 1.243658763416918, 2.5186438960145057]


In [108]:
np.cos(i)

0.6721248293949382

In [109]:
# 给出P_new, Q_new
P_new = np.array([np.cos(Omega)*np.cos(w) - np.sin(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(Omega)*np.cos(w) + np.cos(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(w)*np.sin(i) ])
Q_new = np.array([-np.cos(Omega)*np.sin(w) - np.sin(Omega)*np.cos(w)*np.cos(i),\
                 -np.sin(Omega)*np.sin(w) + np.cos(Omega)*np.cos(w)*np.cos(i),\
                 np.cos(w)*np.sin(i)])

print(P,Q)
print(P_new,Q_new)

[ 0.89875297  0.14062793 -0.41529132] [0.16334348 0.7733904  0.61250482]
[ 0.89869077  0.14033182 -0.41552603] [0.16261318 0.7732856  0.61285099]


In [110]:
# 归算rt, vt
rt = [];vt=[];
for Et_single in Et:
    rt_single = a*(np.cos(Et_single) - e)*P_new + a*np.sqrt(1 - e**2)*np.sin(Et_single)*Q_new
    rt_single_norm = np.sqrt( np.dot(rt_single,rt_single) )
    rt.append(rt_single)
    
    vt_single = -a**2*n/rt_single_norm*np.sin(Et_single)*P_new + a**2*n/rt_single_norm*np.sqrt(1 - e**2)*np.cos(Et_single)*Q_new
    vt_single_norm = np.sqrt( np.dot(vt_single,vt_single) )
    vt.append(rt_single)
    
    print(rt_single_norm/10**3,vt_single_norm/10**3)
    
rt,vt

7625.807231972519 6.113191800675767
6695.287137968549 7.20262219407637
4503.783332355857 10.475030057641632
5397.478882138164 8.970135805083444
7309.178894864184 6.472457874704945


([array([-6839663.32454   , -1007352.53277392,  3218257.66850027]),
  array([-4735748.30345643, -4547004.3273072 , -1313129.66366854]),
  array([ 2541919.70988409, -1838287.0422574 , -3231626.40841693]),
  array([1076454.00398044, 4201412.78216185, 3212811.1824894 ]),
  array([-5310469.547858  ,  1655721.99642122,  4741475.92727916])],
 [array([-6839663.32454   , -1007352.53277392,  3218257.66850027]),
  array([-4735748.30345643, -4547004.3273072 , -1313129.66366854]),
  array([ 2541919.70988409, -1838287.0422574 , -3231626.40841693]),
  array([1076454.00398044, 4201412.78216185, 3212811.1824894 ]),
  array([-5310469.547858  ,  1655721.99642122,  4741475.92727916])])

In [111]:
# 计算方向矢量L：
P_earth = rt
station_earth = np.array([-0.81000791, -0.25188507,  0.52786921])*R_earth

P_station = P_earth - station_earth

L_forecast_total = []
for P_station_single in P_station:

    L_forecast_total.append( P_station_single/np.sqrt(np.dot(P_station_single,P_station_single)) )

L_forecast_total

[array([-0.93905091,  0.33410449, -0.08097885]),
 array([ 0.07666544, -0.53098409, -0.84390657]),
 array([ 0.75941789, -0.02302437, -0.65019562]),
 array([ 0.73182137,  0.68126845, -0.01762886]),
 array([-0.04231067,  0.92024513,  0.38904845])]

In [112]:
L = [-0.9152477093203553, -0.0501154444748697, -0.39976252051552885]
L = [-0.921558,0.385984,-0.041788]
L

[-0.921558, 0.385984, -0.041788]